# Scratchwork

This notebook is only used for trying out ideas.

In [1]:
import algebras as alg
import json
import os
import numpy as np
import itertools as it
from pprint import pprint

from itertools import combinations, permutations, product

In [2]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups for Testing

### D4 - Dihedral Group on 4 Vertices

In [3]:
d4_path = os.path.join(alg_dir, "d4_dihedral_group_on_4_vertices.json")
!cat {d4_path}

{"type": "Group",
 "name": "D_4",
 "description": "Dihedral group on four vertices",
 "element_names": ["e", "r", "r^2", "r^3", "f", "fr", "r^2f", "rf"],
 "alt_elem_names": ["()", "(0 1 2 3)", "(0 2)(1 3)", "(0 3 2 1)",
                    "(0 1)(2 3)", "(1 3)", "(0 3)(1 2)", "(0 2)"],
 "mult_table": [[0, 1, 2, 3, 4, 5, 6, 7],
                [1, 2, 3, 0, 7, 4, 5, 6],
                [2, 3, 0, 1, 6, 7, 4, 5],
                [3, 0, 1, 2, 5, 6, 7, 4],
                [4, 5, 6, 7, 0, 1, 2, 3],
                [5, 6, 7, 4, 3, 0, 1, 2],
                [6, 7, 4, 5, 2, 3, 0, 1],
                [7, 4, 5, 6, 1, 2, 3, 0]]
}

In [4]:
d4 = alg.Group(d4_path)
d4.print_info()


Group : D_4 : Dihedral group on four vertices
  Element Names: ['e', 'r', 'r^2', 'r^3', 'f', 'fr', 'r^2f', 'rf']
  Is Abelian? False
  Inverses:  (** - indicates that it is its own inverse)
    inv(e) = e   **
    inv(r) = r^3 
    inv(r^2) = r^2   **
    inv(r^3) = r 
    inv(f) = f   **
    inv(fr) = fr   **
    inv(r^2f) = r^2f   **
    inv(rf) = rf   **
Element Orders:
{1: ['e'], 2: ['r^2', 'f', 'fr', 'r^2f', 'rf'], 4: ['r', 'r^3']}
  Is associative? True
  Cayley Table:
[['e', 'r', 'r^2', 'r^3', 'f', 'fr', 'r^2f', 'rf'],
 ['r', 'r^2', 'r^3', 'e', 'rf', 'f', 'fr', 'r^2f'],
 ['r^2', 'r^3', 'e', 'r', 'r^2f', 'rf', 'f', 'fr'],
 ['r^3', 'e', 'r', 'r^2', 'fr', 'r^2f', 'rf', 'f'],
 ['f', 'fr', 'r^2f', 'rf', 'e', 'r', 'r^2', 'r^3'],
 ['fr', 'r^2f', 'rf', 'f', 'r^3', 'e', 'r', 'r^2'],
 ['r^2f', 'rf', 'f', 'fr', 'r^2', 'r^3', 'e', 'r'],
 ['rf', 'f', 'fr', 'r^2f', 'r', 'r^2', 'r^3', 'e']]


### V4 - Klein 4 Group

In [5]:
v4_path = os.path.join(alg_dir, "v4_klein_4_group.json")
!cat {v4_path}

{"type": "Group",
 "name": "V4",
 "description": "Klein-4 group",
 "element_names": ["e", "h", "v", "hv"],
 "mult_table": [[0, 1, 2, 3],
                [1, 0, 3, 2],
                [2, 3, 0, 1],
                [3, 2, 1, 0]]
}


In [6]:
v4 = alg.Group(v4_path)
v4.print_info()


Group : V4 : Klein-4 group
  Element Names: ['e', 'h', 'v', 'hv']
  Is Abelian? True
  Inverses:  (** - indicates that it is its own inverse)
    inv(e) = e   **
    inv(h) = h   **
    inv(v) = v   **
    inv(hv) = hv   **
Element Orders:
{1: ['e'], 2: ['h', 'v', 'hv']}
  Is associative? True
  Cayley Table:
[['e', 'h', 'v', 'hv'],
 ['h', 'e', 'hv', 'v'],
 ['v', 'hv', 'e', 'h'],
 ['hv', 'v', 'h', 'e']]


In [7]:
z2 = alg.generate_cyclic_group(2)
z2

Group('Z2',
'Autogenerated cyclic group of order 2',
['e', 'a'],
[[0, 1], [1, 0]]) 

In [8]:
z2_x_z2 = z2 * z2
z2_x_z2

Group('Z2_x_Z2',
'Direct product of Z2 & Z2',
['e:e', 'e:a', 'a:e', 'a:a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [9]:
z4 = alg.generate_cyclic_group(4)
z4

Group('Z4',
'Autogenerated cyclic group of order 4',
['e', 'a', 'a^2', 'a^3'],
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) 

## Changing the Order of Elements

<b>Observation</b>: If the group is abelian, then changing the order of the elements does <b>not</b> affect the order of the indices in the <i>mult_table</i>.  <b>Why?</b>

In [10]:
# new_elem_order = v4.element_names
# new_v4_elem_order = ['e', 'v', 'h', 'hv']
new_v4_elem_order = ["e", "hv", "h", "v"]

In [11]:
#table_from_reordered_elements(v4, new_v4_elem_order)
v4_reordered = v4.reorder_elements(new_v4_elem_order)
v4_reordered

Group('V4_REORDERED',
'Klein-4 group (elements reordered)',
['e', 'hv', 'h', 'v'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [12]:
pprint(v4_reordered.mult_table_with_names())

[['e', 'hv', 'h', 'v'],
 ['hv', 'e', 'v', 'h'],
 ['h', 'v', 'e', 'hv'],
 ['v', 'h', 'hv', 'e']]


In [13]:
v4_reordered.mult_table

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 0, 1],
       [3, 2, 1, 0]])

In [14]:
pprint(v4.mult_table_with_names())

[['e', 'h', 'v', 'hv'],
 ['h', 'e', 'hv', 'v'],
 ['v', 'hv', 'e', 'h'],
 ['hv', 'v', 'h', 'e']]


In [15]:
v4.mult_table

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 0, 1],
       [3, 2, 1, 0]])

In [16]:
new_d4_elem_order = ["e", "r", "r^2", "f", "r^3", "fr", "r^2f", "rf"]

In [17]:
d4_reordered = d4.reorder_elements(new_d4_elem_order)
d4_reordered

Group('D_4_REORDERED',
'Dihedral group on four vertices (elements reordered)',
['e', 'r', 'r^2', 'f', 'r^3', 'fr', 'r^2f', 'rf'],
[[0, 1, 2, 3, 4, 5, 6, 7], [1, 2, 4, 7, 0, 3, 5, 6], [2, 4, 0, 6, 1, 7, 3, 5], [3, 5, 6, 0, 7, 1, 2, 4], [4, 0, 1, 5, 2, 6, 7, 3], [5, 6, 7, 4, 3, 0, 1, 2], [6, 7, 3, 2, 5, 4, 0, 1], [7, 3, 5, 1, 6, 2, 4, 0]]) 

In [18]:
d4_reordered.mult_table

array([[0, 1, 2, 3, 4, 5, 6, 7],
       [1, 2, 4, 7, 0, 3, 5, 6],
       [2, 4, 0, 6, 1, 7, 3, 5],
       [3, 5, 6, 0, 7, 1, 2, 4],
       [4, 0, 1, 5, 2, 6, 7, 3],
       [5, 6, 7, 4, 3, 0, 1, 2],
       [6, 7, 3, 2, 5, 4, 0, 1],
       [7, 3, 5, 1, 6, 2, 4, 0]])

In [19]:
d4.mult_table

array([[0, 1, 2, 3, 4, 5, 6, 7],
       [1, 2, 3, 0, 7, 4, 5, 6],
       [2, 3, 0, 1, 6, 7, 4, 5],
       [3, 0, 1, 2, 5, 6, 7, 4],
       [4, 5, 6, 7, 0, 1, 2, 3],
       [5, 6, 7, 4, 3, 0, 1, 2],
       [6, 7, 4, 5, 2, 3, 0, 1],
       [7, 4, 5, 6, 1, 2, 3, 0]])

In [20]:
d4.element_orders()

{'e': 1, 'r': 4, 'r^2': 2, 'r^3': 4, 'f': 2, 'fr': 2, 'r^2f': 2, 'rf': 2}

In [21]:
d4_reordered.element_orders()

{'e': 1, 'r': 4, 'r^2': 2, 'f': 2, 'r^3': 4, 'fr': 2, 'r^2f': 2, 'rf': 2}

## Generating Possible Multiplication Tables

In [22]:
tables4 = alg.generate_all_group_tables(4)
tables4

[([0, 1, 2, 3], (1, 0, 3, 2), (2, 3, 0, 1), (3, 2, 1, 0)),
 ([0, 1, 2, 3], (1, 0, 3, 2), (2, 3, 1, 0), (3, 2, 0, 1)),
 ([0, 1, 2, 3], (1, 2, 3, 0), (2, 3, 0, 1), (3, 0, 1, 2)),
 ([0, 1, 2, 3], (1, 3, 0, 2), (2, 0, 3, 1), (3, 2, 1, 0))]

In [23]:
for t in tables4:
    pprint(np.array(t))

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 0, 1],
       [3, 2, 1, 0]])
array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 1, 0],
       [3, 2, 0, 1]])
array([[0, 1, 2, 3],
       [1, 2, 3, 0],
       [2, 3, 0, 1],
       [3, 0, 1, 2]])
array([[0, 1, 2, 3],
       [1, 3, 0, 2],
       [2, 0, 3, 1],
       [3, 2, 1, 0]])


In [24]:
groups4 = alg.tables_to_groups(tables4)
for g in groups4:
    g.pprint()

Group('G0',
'Group 0 of order 4',
['e0', 'a0_1', 'a0_2', 'a0_3'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)
Group('G1',
'Group 1 of order 4',
['e1', 'a1_1', 'a1_2', 'a1_3'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 1, 0], [3, 2, 0, 1]]
)
Group('G2',
'Group 2 of order 4',
['e2', 'a2_1', 'a2_2', 'a2_3'],
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]
)
Group('G3',
'Group 3 of order 4',
['e3', 'a3_1', 'a3_2', 'a3_3'],
[[0, 1, 2, 3], [1, 3, 0, 2], [2, 0, 3, 1], [3, 2, 1, 0]]
)


In [25]:
g0 = groups4[0]
g1 = groups4[1]
g2 = groups4[2]
g3 = groups4[3]

Below, it can be seen that...

* <b>g0</b> is isomorphic to, v4, the Klein-4 group (and to the direct product, z2 * z2)
* <b>g1, g2, & g3</b> are isomorphic to each other and to, z4, the cyclic group of order 4

## G0

In [26]:
g0  # Isomorphic to v4 and z2_x_z2

Group('G0',
'Group 0 of order 4',
['e0', 'a0_1', 'a0_2', 'a0_3'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [27]:
v4.pprint()

Group('V4',
'Klein-4 group',
['e', 'h', 'v', 'hv'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)


In [28]:
g0.isomorphic(v4)

{'a0_1': 'h', 'a0_2': 'v', 'a0_3': 'hv', 'e0': 'e'}

In [29]:
g0.isomorphic(z2_x_z2)

{'a0_1': 'e:a', 'a0_2': 'a:e', 'a0_3': 'a:a', 'e0': 'e:e'}

In [30]:
v4.isomorphic(z2_x_z2)

{'h': 'e:a', 'v': 'a:e', 'hv': 'a:a', 'e': 'e:e'}

## G1

In [31]:
g1

Group('G1',
'Group 1 of order 4',
['e1', 'a1_1', 'a1_2', 'a1_3'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 1, 0], [3, 2, 0, 1]]) 

In [32]:
g1.isomorphic(g0)

False

## G2

In [33]:
g2  # Same as z4, below

Group('G2',
'Group 2 of order 4',
['e2', 'a2_1', 'a2_2', 'a2_3'],
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) 

In [34]:
g2.isomorphic(z4)

{'a2_1': 'a', 'a2_2': 'a^2', 'a2_3': 'a^3', 'e2': 'e'}

In [35]:
g2.isomorphic(g0)

False

In [36]:
g2.isomorphic(g1)

{'a2_1': 'a1_2', 'a2_2': 'a1_1', 'a2_3': 'a1_3', 'e2': 'e1'}

## G3

In [37]:
g3

Group('G3',
'Group 3 of order 4',
['e3', 'a3_1', 'a3_2', 'a3_3'],
[[0, 1, 2, 3], [1, 3, 0, 2], [2, 0, 3, 1], [3, 2, 1, 0]]) 

In [38]:
g3.isomorphic(g0)

False

In [39]:
g3.isomorphic(g1)

{'a3_1': 'a1_2', 'a3_2': 'a1_3', 'a3_3': 'a1_1', 'e3': 'e1'}

In [40]:
g3.isomorphic(g2)

{'a3_1': 'a2_1', 'a3_2': 'a2_3', 'a3_3': 'a2_2', 'e3': 'e2'}

## Groups of Prime Order

In [41]:
order = 5

In [42]:
z5 = alg.generate_cyclic_group(order)

In [43]:
z5.pprint()

Group('Z5',
'Autogenerated cyclic group of order 5',
['e', 'a', 'a^2', 'a^3', 'a^4'],
[[0, 1, 2, 3, 4],
 [1, 2, 3, 4, 0],
 [2, 3, 4, 0, 1],
 [3, 4, 0, 1, 2],
 [4, 0, 1, 2, 3]]
)


In [44]:
tables5 = alg.generate_all_group_tables(order)
len(tables5)

6

In [45]:
groups5 = alg.tables_to_groups(tables5)
for grp in groups5:
    grp.pprint()

Group('G0',
'Group 0 of order 5',
['e0', 'a0_1', 'a0_2', 'a0_3', 'a0_4'],
[[0, 1, 2, 3, 4],
 [1, 2, 3, 4, 0],
 [2, 3, 4, 0, 1],
 [3, 4, 0, 1, 2],
 [4, 0, 1, 2, 3]]
)
Group('G1',
'Group 1 of order 5',
['e1', 'a1_1', 'a1_2', 'a1_3', 'a1_4'],
[[0, 1, 2, 3, 4],
 [1, 2, 4, 0, 3],
 [2, 4, 3, 1, 0],
 [3, 0, 1, 4, 2],
 [4, 3, 0, 2, 1]]
)
Group('G2',
'Group 2 of order 5',
['e2', 'a2_1', 'a2_2', 'a2_3', 'a2_4'],
[[0, 1, 2, 3, 4],
 [1, 3, 0, 4, 2],
 [2, 0, 4, 1, 3],
 [3, 4, 1, 2, 0],
 [4, 2, 3, 0, 1]]
)
Group('G3',
'Group 3 of order 5',
['e3', 'a3_1', 'a3_2', 'a3_3', 'a3_4'],
[[0, 1, 2, 3, 4],
 [1, 3, 4, 2, 0],
 [2, 4, 1, 0, 3],
 [3, 2, 0, 4, 1],
 [4, 0, 3, 1, 2]]
)
Group('G4',
'Group 4 of order 5',
['e4', 'a4_1', 'a4_2', 'a4_3', 'a4_4'],
[[0, 1, 2, 3, 4],
 [1, 4, 0, 2, 3],
 [2, 0, 3, 4, 1],
 [3, 2, 4, 1, 0],
 [4, 3, 1, 0, 2]]
)
Group('G5',
'Group 5 of order 5',
['e5', 'a5_1', 'a5_2', 'a5_3', 'a5_4'],
[[0, 1, 2, 3, 4],
 [1, 4, 3, 0, 2],
 [2, 3, 1, 4, 0],
 [3, 0, 4, 2, 1],
 [4, 2, 0, 1, 3]]
)


In [46]:
# No two tables in groups5 are the same:

np_tables5 = [g.mult_table for g in groups5]

for combo in combinations(groups5, 2):
    if combo[0] == combo[1]:
        print(combo[0])

In [47]:
# And yet, six of the groups are abelian:

abelian_tables5 = [g for g in groups5 if g.abelian()]
abelian_tables5

[Group('G0',
 'Group 0 of order 5',
 ['e0', 'a0_1', 'a0_2', 'a0_3', 'a0_4'],
 [[0, 1, 2, 3, 4], [1, 2, 3, 4, 0], [2, 3, 4, 0, 1], [3, 4, 0, 1, 2], [4, 0, 1, 2, 3]]) ,
 Group('G1',
 'Group 1 of order 5',
 ['e1', 'a1_1', 'a1_2', 'a1_3', 'a1_4'],
 [[0, 1, 2, 3, 4], [1, 2, 4, 0, 3], [2, 4, 3, 1, 0], [3, 0, 1, 4, 2], [4, 3, 0, 2, 1]]) ,
 Group('G2',
 'Group 2 of order 5',
 ['e2', 'a2_1', 'a2_2', 'a2_3', 'a2_4'],
 [[0, 1, 2, 3, 4], [1, 3, 0, 4, 2], [2, 0, 4, 1, 3], [3, 4, 1, 2, 0], [4, 2, 3, 0, 1]]) ,
 Group('G3',
 'Group 3 of order 5',
 ['e3', 'a3_1', 'a3_2', 'a3_3', 'a3_4'],
 [[0, 1, 2, 3, 4], [1, 3, 4, 2, 0], [2, 4, 1, 0, 3], [3, 2, 0, 4, 1], [4, 0, 3, 1, 2]]) ,
 Group('G4',
 'Group 4 of order 5',
 ['e4', 'a4_1', 'a4_2', 'a4_3', 'a4_4'],
 [[0, 1, 2, 3, 4], [1, 4, 0, 2, 3], [2, 0, 3, 4, 1], [3, 2, 4, 1, 0], [4, 3, 1, 0, 2]]) ,
 Group('G5',
 'Group 5 of order 5',
 ['e5', 'a5_1', 'a5_2', 'a5_3', 'a5_4'],
 [[0, 1, 2, 3, 4], [1, 4, 3, 0, 2], [2, 3, 1, 4, 0], [3, 0, 4, 2, 1], [4, 2, 0, 1, 3]]) 

In [48]:
tbl0 = tables5[0]
print(tbl0)

([0, 1, 2, 3, 4], (1, 2, 3, 4, 0), (2, 3, 4, 0, 1), (3, 4, 0, 1, 2), (4, 0, 1, 2, 3))


In [49]:
alg.associative_table(tbl0)

True

In [50]:
alg.generate_all_group_tables(4)

[([0, 1, 2, 3], (1, 0, 3, 2), (2, 3, 0, 1), (3, 2, 1, 0)),
 ([0, 1, 2, 3], (1, 0, 3, 2), (2, 3, 1, 0), (3, 2, 0, 1)),
 ([0, 1, 2, 3], (1, 2, 3, 0), (2, 3, 0, 1), (3, 0, 1, 2)),
 ([0, 1, 2, 3], (1, 3, 0, 2), (2, 0, 3, 1), (3, 2, 1, 0))]

<b>IMPORTANT NOTE:</b>The computation of all group tables of order 6 currently takes between 4.5 & 6 hours, so the resuls have been saved in a file, tables5.py.  The file, groups5.py, was also saved.

In [51]:
%run -i '../misc/tables6.py'

In [52]:
tables6
len(tables6)

80

In [53]:
groups6 = alg.tables_to_groups(tables6)
groups6

[Group('G0',
 'Group 0 of order 6',
 ['e0', 'a0_1', 'a0_2', 'a0_3', 'a0_4', 'a0_5'],
 [[0, 1, 2, 3, 4, 5], [1, 0, 3, 2, 5, 4], [2, 3, 4, 5, 0, 1], [3, 2, 5, 4, 1, 0], [4, 5, 0, 1, 2, 3], [5, 4, 1, 0, 3, 2]]) ,
 Group('G1',
 'Group 1 of order 6',
 ['e1', 'a1_1', 'a1_2', 'a1_3', 'a1_4', 'a1_5'],
 [[0, 1, 2, 3, 4, 5], [1, 0, 3, 2, 5, 4], [2, 3, 4, 5, 1, 0], [3, 2, 5, 4, 0, 1], [4, 5, 1, 0, 3, 2], [5, 4, 0, 1, 2, 3]]) ,
 Group('G2',
 'Group 2 of order 6',
 ['e2', 'a2_1', 'a2_2', 'a2_3', 'a2_4', 'a2_5'],
 [[0, 1, 2, 3, 4, 5], [1, 0, 3, 2, 5, 4], [2, 3, 5, 4, 0, 1], [3, 2, 4, 5, 1, 0], [4, 5, 0, 1, 3, 2], [5, 4, 1, 0, 2, 3]]) ,
 Group('G3',
 'Group 3 of order 6',
 ['e3', 'a3_1', 'a3_2', 'a3_3', 'a3_4', 'a3_5'],
 [[0, 1, 2, 3, 4, 5], [1, 0, 3, 2, 5, 4], [2, 3, 5, 4, 1, 0], [3, 2, 4, 5, 0, 1], [4, 5, 1, 0, 2, 3], [5, 4, 0, 1, 3, 2]]) ,
 Group('G4',
 'Group 4 of order 6',
 ['e4', 'a4_1', 'a4_2', 'a4_3', 'a4_4', 'a4_5'],
 [[0, 1, 2, 3, 4, 5], [1, 0, 3, 2, 5, 4], [2, 4, 0, 5, 1, 3], [3, 5, 1, 4, 

In [54]:
not_abelian6 = [g.name for g in groups6 if not g.abelian()]
print(not_abelian6)

['G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G27', 'G28', 'G43', 'G46', 'G58', 'G61', 'G73', 'G76']


In [55]:
iso_to_g4 = [g for g in groups6 if groups6[4].isomorphic(g)]
len(iso_to_g4)

20

In [56]:
iso_to_g0 = [g for g in groups6 if groups6[0].isomorphic(g)]
len(iso_to_g0)

60

In [57]:
g0 = groups6[0]  # abelian
g4 = groups6[4]  # not abelian

In [58]:
g0.pprint()

Group('G0',
'Group 0 of order 6',
['e0', 'a0_1', 'a0_2', 'a0_3', 'a0_4', 'a0_5'],
[[0, 1, 2, 3, 4, 5],
 [1, 0, 3, 2, 5, 4],
 [2, 3, 4, 5, 0, 1],
 [3, 2, 5, 4, 1, 0],
 [4, 5, 0, 1, 2, 3],
 [5, 4, 1, 0, 3, 2]]
)


In [59]:
g4.pprint()

Group('G4',
'Group 4 of order 6',
['e4', 'a4_1', 'a4_2', 'a4_3', 'a4_4', 'a4_5'],
[[0, 1, 2, 3, 4, 5],
 [1, 0, 3, 2, 5, 4],
 [2, 4, 0, 5, 1, 3],
 [3, 5, 1, 4, 0, 2],
 [4, 2, 5, 0, 3, 1],
 [5, 3, 4, 1, 2, 0]]
)


In [60]:
g0.proper_subgroups()

[Group('G0_subgroup_0',
 'Subgroup of: Group 0 of order 6',
 ['e0', 'a0_1'],
 [[0, 1], [1, 0]]) ,
 Group('G0_subgroup_1',
 'Subgroup of: Group 0 of order 6',
 ['e0', 'a0_2', 'a0_4'],
 [[0, 1, 2], [1, 2, 0], [2, 0, 1]]) ]

In [61]:
g4.proper_subgroups()

[Group('G4_subgroup_0',
 'Subgroup of: Group 4 of order 6',
 ['e4', 'a4_5'],
 [[0, 1], [1, 0]]) ,
 Group('G4_subgroup_1',
 'Subgroup of: Group 4 of order 6',
 ['e4', 'a4_2'],
 [[0, 1], [1, 0]]) ,
 Group('G4_subgroup_2',
 'Subgroup of: Group 4 of order 6',
 ['e4', 'a4_3', 'a4_4'],
 [[0, 1, 2], [1, 2, 0], [2, 0, 1]]) ,
 Group('G4_subgroup_3',
 'Subgroup of: Group 4 of order 6',
 ['e4', 'a4_1'],
 [[0, 1], [1, 0]]) ]

### Turn isomorphisms into permutations

In [62]:
groups4

[Group('G0',
 'Group 0 of order 4',
 ['e0', 'a0_1', 'a0_2', 'a0_3'],
 [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) ,
 Group('G1',
 'Group 1 of order 4',
 ['e1', 'a1_1', 'a1_2', 'a1_3'],
 [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 1, 0], [3, 2, 0, 1]]) ,
 Group('G2',
 'Group 2 of order 4',
 ['e2', 'a2_1', 'a2_2', 'a2_3'],
 [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) ,
 Group('G3',
 'Group 3 of order 4',
 ['e3', 'a3_1', 'a3_2', 'a3_3'],
 [[0, 1, 2, 3], [1, 3, 0, 2], [2, 0, 3, 1], [3, 2, 1, 0]]) ]

In [63]:
for combo in combinations(groups4,2):
    mp = combo[0].isomorphic(combo[1])
    print(combo[0].name, combo[1].name, mp)

G0 G1 False
G0 G2 False
G0 G3 False
G1 G2 {'a1_1': 'a2_2', 'a1_2': 'a2_1', 'a1_3': 'a2_3', 'e1': 'e2'}
G1 G3 {'a1_1': 'a3_3', 'a1_2': 'a3_1', 'a1_3': 'a3_2', 'e1': 'e3'}
G2 G3 {'a2_1': 'a3_1', 'a2_2': 'a3_3', 'a2_3': 'a3_2', 'e2': 'e3'}


In [64]:
def get_integer_form(elem_list):
    """For an element list like ['e1', 'a1_2', 'a1_1', 'a1_3'],
    return the integer 213, i.e., the 'subscripts' of the elements that
    follow the identity element."""
    return int(''.join(map(lambda x: x.split("_")[1], elem_list[1:])))

In [65]:
z4_iso_groups4 = [z4.isomorphic(grp) for grp in groups4[1:]]
z4_iso_groups4

[{'a': 'a1_2', 'a^2': 'a1_1', 'a^3': 'a1_3', 'e': 'e1'},
 {'a': 'a2_1', 'a^2': 'a2_2', 'a^3': 'a2_3', 'e': 'e2'},
 {'a': 'a3_1', 'a^2': 'a3_3', 'a^3': 'a3_2', 'e': 'e3'}]

In [66]:
# Get values from the iso mapping in the same order as the elements in z4

test_iso = z4_iso_groups4[0]
[test_iso[elem] for elem in z4.element_names]

['e1', 'a1_2', 'a1_1', 'a1_3']

In [67]:
get_integer_form([test_iso[elem] for elem in z4.element_names])

213

In [68]:
[get_integer_form([iso[elem] for elem in z4.element_names]) for iso in z4_iso_groups4]

[213, 123, 132]

In [69]:
def get_int_forms(ref_group, isomorphisms):
    return [get_integer_form([iso[elem] for elem in ref_group.element_names])
            for iso in isomorphisms]

In [70]:
get_int_forms(z4, z4_iso_groups4)

[213, 123, 132]

In [71]:
g1_iso_groups4 = [g1.isomorphic(grp) for grp in groups4[1:]]
g1_iso_groups4

get_int_forms(g1, g1_iso_groups4)

[123, 213, 312]

In [72]:
z5

Group('Z5',
'Autogenerated cyclic group of order 5',
['e', 'a', 'a^2', 'a^3', 'a^4'],
[[0, 1, 2, 3, 4], [1, 2, 3, 4, 0], [2, 3, 4, 0, 1], [3, 4, 0, 1, 2], [4, 0, 1, 2, 3]]) 

In [73]:
groups5

[Group('G0',
 'Group 0 of order 5',
 ['e0', 'a0_1', 'a0_2', 'a0_3', 'a0_4'],
 [[0, 1, 2, 3, 4], [1, 2, 3, 4, 0], [2, 3, 4, 0, 1], [3, 4, 0, 1, 2], [4, 0, 1, 2, 3]]) ,
 Group('G1',
 'Group 1 of order 5',
 ['e1', 'a1_1', 'a1_2', 'a1_3', 'a1_4'],
 [[0, 1, 2, 3, 4], [1, 2, 4, 0, 3], [2, 4, 3, 1, 0], [3, 0, 1, 4, 2], [4, 3, 0, 2, 1]]) ,
 Group('G2',
 'Group 2 of order 5',
 ['e2', 'a2_1', 'a2_2', 'a2_3', 'a2_4'],
 [[0, 1, 2, 3, 4], [1, 3, 0, 4, 2], [2, 0, 4, 1, 3], [3, 4, 1, 2, 0], [4, 2, 3, 0, 1]]) ,
 Group('G3',
 'Group 3 of order 5',
 ['e3', 'a3_1', 'a3_2', 'a3_3', 'a3_4'],
 [[0, 1, 2, 3, 4], [1, 3, 4, 2, 0], [2, 4, 1, 0, 3], [3, 2, 0, 4, 1], [4, 0, 3, 1, 2]]) ,
 Group('G4',
 'Group 4 of order 5',
 ['e4', 'a4_1', 'a4_2', 'a4_3', 'a4_4'],
 [[0, 1, 2, 3, 4], [1, 4, 0, 2, 3], [2, 0, 3, 4, 1], [3, 2, 4, 1, 0], [4, 3, 1, 0, 2]]) ,
 Group('G5',
 'Group 5 of order 5',
 ['e5', 'a5_1', 'a5_2', 'a5_3', 'a5_4'],
 [[0, 1, 2, 3, 4], [1, 4, 3, 0, 2], [2, 3, 1, 4, 0], [3, 0, 4, 2, 1], [4, 2, 0, 1, 3]]) 

In [74]:
z5_iso_groups5 = [z5.isomorphic(grp) for grp in groups5]
z5_iso_groups5

[{'a': 'a0_1', 'a^2': 'a0_2', 'a^3': 'a0_3', 'a^4': 'a0_4', 'e': 'e0'},
 {'a': 'a1_1', 'a^2': 'a1_2', 'a^3': 'a1_4', 'a^4': 'a1_3', 'e': 'e1'},
 {'a': 'a2_1', 'a^2': 'a2_3', 'a^3': 'a2_4', 'a^4': 'a2_2', 'e': 'e2'},
 {'a': 'a3_1', 'a^2': 'a3_3', 'a^3': 'a3_2', 'a^4': 'a3_4', 'e': 'e3'},
 {'a': 'a4_1', 'a^2': 'a4_4', 'a^3': 'a4_3', 'a^4': 'a4_2', 'e': 'e4'},
 {'a': 'a5_1', 'a^2': 'a5_4', 'a^3': 'a5_2', 'a^4': 'a5_3', 'e': 'e5'}]

In [75]:
get_int_forms(z5, z5_iso_groups5)

[1234, 1243, 1342, 1324, 1432, 1423]